## Bài 1

In [7]:
# ==============================================================
# Bài 1a – User Cold-start Recommendation
# ==============================================================
import pandas as pd
import numpy as np

# Giả lập dữ liệu ví dụ nhỏ (vì đề không cho sẵn file)
# np.random.seed(42)
# users = np.arange(1, 101)       # 100 users
# songs = np.arange(1, 21)        # 20 songs
# data = []
# for u in users:
#     rated = np.random.choice(songs, np.random.randint(5, 10), replace=False)
#     for s in rated:
#         data.append([u, s, np.random.randint(1, 6)])

# df = pd.DataFrame(data, columns=['user_id', 'song_id', 'rating'])
# Nếu bạn có file thật: df = pd.read_csv("songs.csv")
df = pd.read_csv("data/data.csv")


# --- Tính trung bình & số rating cho mỗi bài hát ---
agg = df.groupby('song_id')['rating'].agg(['count', 'mean']).reset_index()
global_mean = df['rating'].mean()
k = 50

# --- Công thức Bayesian Weighted Rating ---
agg['score'] = (agg['count']/(agg['count']+k))*agg['mean'] + (k/(agg['count']+k))*global_mean

# --- Lấy Top 5 bài hát có điểm cao nhất ---
top5_coldstart = agg.sort_values('score', ascending=False).head(5)
print("=== 🧠 Top 5 bài hát nên nghe cho user mới (Cold-start) ===")
print(top5_coldstart[['song_id', 'count', 'mean', 'score']])


=== 🧠 Top 5 bài hát nên nghe cho user mới (Cold-start) ===
    song_id  count      mean     score
17       18     18  3.555556  3.150386
14       15     31  3.354839  3.138596
6         7     14  3.571429  3.128535
9        10     16  3.500000  3.124640
3         4     19  3.368421  3.104728


In [5]:
# ==============================================================
# Bài 1b – User đã nghe bài hát STT = 1
# ==============================================================
# Tạo ma trận nhị phân user-song
pivot = pd.crosstab(df['user_id'], df['song_id'])

# Tính số lần 2 bài hát được nghe cùng nhau
co_occurrence = pivot.T.dot(pivot)

# Loại bỏ chính nó
np.fill_diagonal(co_occurrence.values, 0)

# Lấy hàng tương ứng với bài hát 1
if 1 in co_occurrence.index:
    song1_related = co_occurrence.loc[1].sort_values(ascending=False).head(5)
    print("\n=== 🎧 Top 5 bài hát thường được nghe cùng với bài hát 1 ===")
    print(song1_related)
else:
    print("⚠️ Bài hát 1 chưa có trong dữ liệu.")



=== 🎧 Top 5 bài hát thường được nghe cùng với bài hát 1 ===
song_id
12    18
14    11
11    10
15     9
17     9
Name: 1, dtype: int64


=== 🆕 Khi thêm bài hát mới (item cold-start) ===

→ Giải pháp:
1. Dùng Content-based Filtering (thể loại, ca sĩ, nhịp điệu…)
2. Dùng Hybrid model kết hợp CF + metadata
3. Warm-start bằng rating ban đầu
4. Nếu chưa có gì: fallback theo độ phổ biến trung bình

### Thêm mới:

In [10]:
agg = df.groupby('song_id')['rating'].agg(['count','mean']).reset_index()
k=50; global_mean=df['rating'].mean()
agg['score']=(agg['count']/(agg['count']+k))*agg['mean']+(k/(agg['count']+k))*global_mean
top5 = agg.sort_values('score',ascending=False).head(5)
top5

,song_id,count,mean,score
17,18,18,3.555556,3.150386
14,15,31,3.354839,3.138596
6,7,14,3.571429,3.128535
9,10,16,3.500000,3.124640
3,4,19,3.368421,3.104728


In [ ]:
# df['timestamp']=pd.to_datetime(df['timestamp'])
# df_recent=df[df['timestamp']>='2025-09-01']
# agg=df_recent.groupby('song_id')['rating'].agg(['count','mean'])
# k=50; global_mean=df['rating'].mean()

In [ ]:
def recommend_coldstart(df, id_col='user_id', item_col='song_id', rating_col='rating', k=50):
    global_mean = df[rating_col].mean()
    agg = df.groupby(item_col)[rating_col].agg(['count','mean']).reset_index()
    agg['score'] = (agg['count']/(agg['count']+k))*agg['mean'] + (k/(agg['count']+k))*global_mean
    return agg.sort_values('score', ascending=False).head(5)

def recommend_cooccurrence(df, id_col='user_id', item_col='song_id', base_item=1):
    pivot = pd.crosstab(df[id_col], df[item_col])
    co = pivot.T.dot(pivot)
    np.fill_diagonal(co.values, 0)
    return co.loc[base_item].sort_values(ascending=False).head(5)



# ==============================================================
# Cách dùng hàm
print("\n=== 🧠 Top 5 bài hát nên nghe cho user mới (Cold-start) ===")
print(recommend_coldstart(df))
print("\n=== 🎧 Top 5 bài hát thường được nghe cùng với bài hát 3 ===")
print(recommend_cooccurrence(df, base_item=3))


=== 🧠 Top 5 bài hát nên nghe cho user mới (Cold-start) ===
    song_id  count      mean     score
17       18     18  3.555556  3.150386
14       15     31  3.354839  3.138596
6         7     14  3.571429  3.128535
9        10     16  3.500000  3.124640
3         4     19  3.368421  3.104728

=== 🎧 Top 5 bài hát thường được nghe cùng với bài hát 1 ===
song_id
12    15
1      7
13     7
15     7
19     6
Name: 3, dtype: int64


| Dạng câu hỏi               | Nhận dạng       | Viết đáp án                         |
| -------------------------- | --------------- | ----------------------------------- |
| “User mới, chưa có rating” | Cold-start user | Dùng độ phổ biến (Bayesian average) |
| “User đã nghe 1 bài”       | Co-occurrence   | Dùng count của bài hát đồng nghe    |
| “Thêm bài mới”             | Item cold-start | Dùng content-based hoặc hybrid      |
| “So sánh 2 phương pháp”    | Lý thuyết       | Nêu ưu-nhược điểm và khi nào dùng   |


✅ Kết luận tổng:

Dạng bài Cold-Start luôn xoay quanh 3 hướng chính:

- Popularity-based: khi user mới

- Co-occurrence-based: khi user có ít hành vi

- Content-based / Hybrid: khi item mới

Hãy nhớ 3 công thức chính:

- Bayesian Average

- Co-occurrence Count

- Jaccard / Cosine Similarity